In [1]:
import pprint

import os
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
drawer = mp.solutions.drawing_utils
VisionRunningMode = mp.tasks.vision.RunningMode

In [3]:
# base options for hand and pose detection models
hand_base_options = python.BaseOptions(model_asset_path="../tasks/hand_landmarker.task")
pose_base_options = python.BaseOptions(model_asset_path="../tasks/pose_landmarker.task")

In [4]:
# options for hand detection
hand_options = vision.HandLandmarkerOptions(
    base_options=hand_base_options,
    num_hands=2,
    min_hand_detection_confidence=0.6,
    min_hand_presence_confidence=0.6,
    min_tracking_confidence=0.6,
    running_mode=VisionRunningMode.IMAGE,
)

# options for pose detection
pose_options = vision.PoseLandmarkerOptions(
    base_options=pose_base_options,
    output_segmentation_masks=True,
    min_pose_detection_confidence=0.6,
    min_pose_presence_confidence=0.6,
    min_tracking_confidence=0.6,
    running_mode=VisionRunningMode.IMAGE,
)

# create detectors
hand_detector = vision.HandLandmarker.create_from_options(hand_options)
pose_detector = vision.PoseLandmarker.create_from_options(pose_options)

In [5]:
empty_hand_landmark = np.zeros((2, 21, 3))  # right hand and left hand
empty_pose_landmark = np.zeros(33 * 3)

def to_landmark_data(
    hand_results: vision.HandLandmarkerResult, pose_results: vision.PoseLandmarkerResult
):
    """
    Extract keypoints from pose and hand results for dataset creation.
    """
    pose_landmark = empty_pose_landmark
    hand_landmark = empty_hand_landmark

    if pose_results.pose_world_landmarks:
        pose_landmark = np.array(
            [[lm.x, lm.y, lm.z] for lm in pose_results.pose_world_landmarks[0]]
        ).flatten()

    # if no hand results are available, return the empty hand keypoints
    # and concatenate it with face and pose keypoints
    if not hand_results:
        return np.concatenate([pose_landmark, hand_landmark.flatten()])

    # iterate over the detected hand landmarks
    for index, hlm in enumerate(hand_results.hand_world_landmarks):
        # determine the hand index (0 for right hand, 1 for left hand) using handedness information
        handedness = hand_results.handedness[index][0].index

        # extract the keypoints for the current hand and assign them to the appropriate index
        hand_landmark[handedness] = np.array([[lm.x, lm.y, lm.z] for lm in hlm])

    return np.concatenate([pose_landmark, hand_landmark.flatten()])

LandmarkList = landmark_pb2.NormalizedLandmarkList  # aliases for landmark types
NormalizedLandmark = landmark_pb2.NormalizedLandmark  # aliases for landmark types


def to_landmark_list(landmarks):
    """
    Create a LandmarkList from a list of landmarks or fill with empty values if no landmarks are provided.
    """
    return LandmarkList(
        landmark=([NormalizedLandmark(x=lm.x, y=lm.y, z=lm.z) for lm in landmarks])
    )


empty_pose_landmarks = to_landmark_list(
    [NormalizedLandmark(x=0.0, y=0.0, z=0.0) for _ in range(33 * 4)]
)

empty_hand_landmarks = to_landmark_list(
    [NormalizedLandmark(x=0.0, y=0.0, z=0.0) for _ in range(21 * 3)]
)


def to_drawing_landmark(hand_results, pose_results):
    """
    Convert pose and hand landmarks to LandmarkList for drawing.
    """
    pose_landmarks = (
        to_landmark_list(pose_results.pose_landmarks[0])
        if pose_results.pose_landmarks
        else empty_pose_landmarks
    )

    hand_landmarks = [empty_hand_landmarks, empty_hand_landmarks]

    if not hand_results:
        return pose_landmarks, None

    # iterate over the detected hand landmarks
    for index, hand_landmark in enumerate(hand_results.hand_landmarks):
        # determine the hand index (0 for right hand, 1 for left hand) using handedness information
        handedness = hand_results.handedness[index][0].index

        # extract the keypoints for the current hand and assign them to the appropriate index
        hand_landmarks[handedness] = to_landmark_list(hand_landmark)

    return pose_landmarks, hand_landmarks


def draw_landmark(image, hand_landmarks, pose_landmarks):
    """
    Draw detected landmarks on the image.
    """
    drawer.draw_landmarks(
        image,
        pose_landmarks,
        mp.solutions.pose.POSE_CONNECTIONS,
        drawer.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=3),
        drawer.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2),
    )

    if not hand_landmarks:
        return

    for hand_landmarks in hand_landmarks:
        drawer.draw_landmarks(
            image,
            hand_landmarks,
            mp.solutions.hands.HAND_CONNECTIONS,
            drawer.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=2),
            drawer.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2),
        )

In [6]:
ACTIONS = np.array([
    'apa', 'aku', 'kamu'
])

In [7]:
def load_model(model_version=None):
    model_dir = "../../storage/models/keras"
    prefix = "signdeafspeech_sds_v_"

    if model_version:
        version = f"{prefix}{model_version}.keras"
        ks_file = os.path.join(model_dir, version)

        model = tf.keras.models.load_model(ks_file)

        return version, model

    model_files = os.listdir(model_dir)

    # filter model files by filename prefix
    versions = [file for file in model_files if file.startswith(prefix)]

    # extract version numbers from filenames
    versions = [file.split("_")[-1] for file in versions]

    # convert version numbers to tuples of integers for comparison
    versions_int = [tuple(map(int, v.split(".")[0])) for v in versions]

    # find the index of the latest version
    latest_index = versions_int.index(max(versions_int))

    # load the latest model
    latest_model_path = model_files[latest_index]

    model = tf.keras.models.load_model(os.path.join(model_dir, latest_model_path))

    return latest_model_path, model


v, model = load_model("006")

In [8]:
f"using model {v}"

'using model signdeafspeech_sds_v_006.keras'

In [9]:
class TFLiteModel:
    def __init__(self, prefix="signdeafspeech_sds_v_"):
        self.model_dir = "../../storage/models/tflite"
        self.prefix = prefix
        self.interpreter = None
        self.input_details = None
        self.output_details = None

        self.input_shape = None
        self.output_shape = None

    def load_model(self, use_latest=True, version=""):
        if not use_latest and not version:
            model_path = os.path.join(self.model_dir, f"{self.prefix}{version}")
        else:
            model_files = os.listdir(self.model_dir)

            # filter model files by filename prefix
            versions = [file for file in model_files if file.startswith(self.prefix)]

            # extract version numbers from filenames
            versions = [file.split("_")[-1] for file in versions]

            # convert version numbers to tuples of integers for comparison
            versions_int = [tuple(map(int, v.split(".")[0])) for v in versions]

            # find the index of the latest version
            latest_index = versions_int.index(max(versions_int))

            # load the latest model
            latest_model_path = model_files[latest_index]
            model_path = os.path.join(self.model_dir, latest_model_path)

        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

        self.input_shape = self.input_details[0]["index"]
        self.output_shape = self.output_details[0]["index"]

        self.print_model_details()

    def print_model_details(self):
        print("Input details:")
        pprint.pprint(self.input_details)
        print()
        print("Output details:")
        pprint.pprint(self.output_details)
        print("=" * 50)

        input_shape = self.input_details[0]["shape"]
        print("Expected input shape:", input_shape)

    def predict(self, input_data):
        self.interpreter.set_tensor(self.input_shape, input_data)
        self.interpreter.invoke()

        result = self.interpreter.get_tensor(self.output_shape)

        return result


tflmodel = TFLiteModel()
tflmodel.load_model()

Input details:
[{'dtype': <class 'numpy.float32'>,
  'index': 0,
  'name': 'serving_default_input_layer_1:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([  1,  60, 225]),
  'shape_signature': array([ -1,  60, 225]),
  'sparsity_parameters': {}}]

Output details:
[{'dtype': <class 'numpy.float32'>,
  'index': 121,
  'name': 'StatefulPartitionedCall_1:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([1, 3]),
  'shape_signature': array([-1,  3]),
  'sparsity_parameters': {}}]
Expected input shape: [  1  60 225]


In [10]:
import random

colors = [
    (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(ACTIONS))
]


def confidence_bar(res, actions, input_frame):
    # create a blank image for the confidence bar
    bar_height = 40 * len(actions)
    bar_width = 300
    confidence_bar_image = np.zeros((bar_height, bar_width, 3), dtype=np.uint8)

    # add text and bars to the confidence bar image
    for num, prob in enumerate(res):
        cv2.putText(
            confidence_bar_image,
            f"{actions[num]}: {int(prob * 100)}%",
            (10, 30 + num * 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1,
            cv2.LINE_AA,
        )
        cv2.rectangle(
            confidence_bar_image,
            (150, 10 + num * 40),
            (150 + int(prob * 150), 30 + num * 40),
            colors[num],
            -1
        )

    # show the confidence bar in a separate window
    cv2.imshow("Confidence Bar", confidence_bar_image)

    return input_frame

In [11]:
import time

In [12]:
cap = cv2.VideoCapture(0)

# set capture properties
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)  # set width to 600 pixels
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)  # set height to 600 pixels
cap.set(cv2.CAP_PROP_FPS, 60)  # set frame rate to 60 FPS

isQuit = False
sequences = []
sequence = []

sentence = []
predictions = []

sequence_length = 60
threshold = 0.9
result = []
skip_word = "_"

while cap.isOpened():
    start = time.time()
    success, image = cap.read()

    if not success:
        print("Ignoring empty camera frame.")
        break

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # convert cv image to mediapipe image format before being
    # passed to face, pose and hand detector
    annotated_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

    hand_results = hand_detector.detect(image=annotated_image)

    pose_results = pose_detector.detect(image=annotated_image)

    keypoints = to_landmark_data(hand_results, pose_results)
    sequences.append(keypoints)
    sequence = np.array(sequences[-60:]).astype(np.float32)

    # pose_landmark, hand_landmark = to_drawing_landmark(hand_results, pose_results)

    # draw_landmark(
    #     image_rgb,
    #     hand_landmarks=hand_landmark,
    #     pose_landmarks=pose_landmark,
    # )

    if len(sequence) == sequence_length:

        # predict the action label based on the sequence of keypoints
        result = tflmodel.predict(np.expand_dims(sequence, axis=0))[0]

        predicted = np.argmax(result) # index of highest result

        predicted_sentence = ACTIONS[predicted]

        # action class with the highest confidence score
        predictions.append(predicted)

        # NOTE: If the current prediction matches the most common prediction over the last 10 frames,
        #       it suggests that the current action is likely intentional and
        #       consistent with recent actions, rather than a momentary anomaly.
        if not (np.unique(predictions[-20:])[0] == predicted):
            pass

        # check if the confidence score of the current prediction index is above the threshold.
        elif result[predicted] < threshold:
            pass

        elif ACTIONS[predicted] == skip_word:
            pass

        elif not sentence or predicted_sentence != sentence[-1]:
            sentence.append(predicted_sentence)

        # limit the length of the recognized action sentence to 5 elements by
        # keeping only the last two elements so it does not exceed the text box
        sentence = sentence[-5:]

        sequences = sequences[-90:]

        predictions = predictions[-20:]

    # overlay the predicted action on the image
    image_rgb = confidence_bar(result, ACTIONS, image_rgb)

    cv2.rectangle(image_rgb, (0, 0), (640, 40), (245, 117, 16), -1)
    cv2.putText(
        image_rgb,
        " ".join(sentence),
        (3, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 255, 255),
        2,
        cv2.LINE_AA,
    )

    cv2.imshow(
        "MediaPipe Detection",
        cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB),
    )

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

    print("exec:", time.time() - start)

cap.release()
cv2.destroyAllWindows()

c:\Users\my\AppData\Local\Programs\Python\Python39\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


exec: 1.0310590267181396
exec: 0.15155577659606934
exec: 0.15026378631591797
exec: 0.11223912239074707
exec: 0.12151622772216797
exec: 0.11494255065917969
exec: 0.10617947578430176
exec: 0.1266777515411377
exec: 0.11809396743774414
exec: 0.12905120849609375
exec: 0.1225130558013916
exec: 0.11912775039672852
exec: 0.10009407997131348
exec: 0.11619091033935547
exec: 0.13642597198486328
exec: 0.2001023292541504
exec: 0.16864967346191406
exec: 0.12254524230957031
exec: 0.07999467849731445
exec: 0.1485307216644287
exec: 0.12248539924621582
exec: 0.12767386436462402
exec: 0.13510966300964355
exec: 0.12915706634521484
exec: 0.12491011619567871
exec: 0.14422297477722168
exec: 0.14456915855407715
exec: 0.11647486686706543
exec: 0.10734105110168457
exec: 0.12973737716674805
exec: 0.09714436531066895
exec: 0.14516997337341309
exec: 0.10120868682861328
exec: 0.1022026538848877
exec: 0.10806965827941895
exec: 0.09733819961547852
exec: 0.1083524227142334
exec: 0.1537935733795166
exec: 0.152622461318

In [29]:
cap.release()
cv2.destroyAllWindows()